In [58]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import time
import json
import concurrent.futures 
from multiprocessing.dummy import Pool as pool
import re
import random
from lxml import etree

## 获取所有分类标签

In [5]:
r = rq.get('https://autumnfish.cn/playlist/catlist')
data = r.text
print(data)

{"code":200,"all":{"name":"全部歌单","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":4,"resourceType":0,"hot":false,"activity":false},"sub":[{"name":"综艺","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":4,"resourceType":0,"hot":true,"activity":false},{"name":"流行","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":1,"resourceType":0,"hot":true,"activity":false},{"name":"影视原声","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":4,"resourceType":0,"hot":true,"activity":false},{"name":"华语","resourceCount":849,"imgId":0,"imgUrl":null,"type":0,"category":0,"resourceType":0,"hot":true,"activity":false},{"name":"清晨","resourceCount":1000,"imgId":0,"imgUrl":null,"type":1,"category":2,"resourceType":0,"hot":false,"activity":false},{"name":"怀旧","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":3,"resourceType":0,"hot":true,"activity":false},{"name":"夜晚","resourceCount":1000,"imgId":0,"imgUrl":null,"type":0,"category":2,"resourc

In [6]:
data_dict = json.loads(data)

In [14]:
data_json = json.dumps(data_dict, indent=3, ensure_ascii=False)

In [15]:
print(data_json)

{
   "code": 200,
   "all": {
      "name": "全部歌单",
      "resourceCount": 1000,
      "imgId": 0,
      "imgUrl": null,
      "type": 0,
      "category": 4,
      "resourceType": 0,
      "hot": false,
      "activity": false
   },
   "sub": [
      {
         "name": "综艺",
         "resourceCount": 1000,
         "imgId": 0,
         "imgUrl": null,
         "type": 0,
         "category": 4,
         "resourceType": 0,
         "hot": true,
         "activity": false
      },
      {
         "name": "流行",
         "resourceCount": 1000,
         "imgId": 0,
         "imgUrl": null,
         "type": 0,
         "category": 1,
         "resourceType": 0,
         "hot": true,
         "activity": false
      },
      {
         "name": "影视原声",
         "resourceCount": 1000,
         "imgId": 0,
         "imgUrl": null,
         "type": 0,
         "category": 4,
         "resourceType": 0,
         "hot": true,
         "activity": false
      },
      {
         "name": "华语",
    

In [21]:
all_subs = data_dict.get("sub")

In [26]:
categories = data_dict.get("categories")
print(categories)

{'0': '语种', '1': '风格', '2': '场景', '3': '情感', '4': '主题'}


In [32]:
categories_dict = {}
all_category = []

In [33]:
for sub in all_subs:
    name = sub.get("name")
    category = sub.get("category")
#     category_key = categories.get(str(category))
    if category not in categories_dict.keys():
        categories_dict[category] = []
    categories_dict[category].append(name)
    # 所有分类标签添加
    all_category.append(name)

In [34]:
all_category

['综艺',
 '流行',
 '影视原声',
 '华语',
 '清晨',
 '怀旧',
 '夜晚',
 '摇滚',
 '欧美',
 '清新',
 'ACG',
 '浪漫',
 '民谣',
 '日语',
 '学习',
 '儿童',
 '电子',
 '韩语',
 '校园',
 '工作',
 '午休',
 '伤感',
 '粤语',
 '游戏',
 '舞曲',
 '说唱',
 '70后',
 '治愈',
 '下午茶',
 '放松',
 '轻音乐',
 '80后',
 '地铁',
 '90后',
 '孤独',
 '驾车',
 '爵士',
 '感动',
 '乡村',
 '网络歌曲',
 '运动',
 '兴奋',
 'KTV',
 'R&B/Soul',
 '旅行',
 '古典',
 '快乐',
 '经典',
 '散步',
 '民族',
 '翻唱',
 '酒吧',
 '安静',
 '吉他',
 '英伦',
 '思念',
 '金属',
 '钢琴',
 '器乐',
 '朋克',
 '蓝调',
 '榜单',
 '00后',
 '雷鬼',
 '世界音乐',
 '拉丁',
 'New Age',
 '古风',
 '后摇',
 'Bossa Nova']

## 获取各个分类下的歌单id

In [21]:
# 读取json中数据
with open(filename) as f:
    my_ip_list = json.load(f)

NameError: name 'filename' is not defined

In [258]:
random.choice(my_ip_list)

'http://113.194.150.76:9999'

In [259]:
list1=[]
play_list_ids=[]
headers = {
    'Referer': 'http://music.163.com/',
    'Host': 'music.163.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.75 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
    }

In [265]:
#通用的获取网站内容的框架
def getHTMLText(url,headers,ip): 
    try:
        proxies = {}
        if ip.split(":")[0] == 'http':
            proxies = {
                'http': ip
            }
        else:
            proxies = {
                'https': ip
            }
        r = rq.get(url,headers=headers,proxies=proxies)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "网络解析错误"

In [266]:
#解析每个广场页
def parse_main(html):
    soup=BeautifulSoup(html,'html.parser')
    c=soup.find_all('li')
    for unit in c:
        try:
            name_url=unit.find('a',{'class':"msk"})#m这里有URL，名字的信息
            # 正则获取歌单的id
            pattern = re.compile(r'\d+')
            id_str = pattern.findall(name_url['href'])
            play_list_ids.append(id_str[0])
        except:
            continue
    print("current counts of ids: " + str(len(play_list_ids)))

In [269]:
#获取首页该分类下面的歌单id
def get_playlist_ids(cat):
    depth=38
    start_url='https://music.163.com/discover/playlist/?order=hot&cat='+cat
    for i in range(depth):
        try:        
            url=start_url+'&limit=35'+'&offset='+str(35*(i))
            # 随机抽取ip
            ip = random.choice(my_ip_list)
            html=getHTMLText(url,headers,ip)
            parse_main(html)
            random_time = random.random()
            time.sleep(random_time * 4)
        except:
            print('失败')
            continue
    print(cat + " 标签爬取完毕")
    print("current counts of ids: " + str(len(play_list_ids)))
    print("========================")

In [270]:
# 根据标签爬取所有歌单id
for item in all_category:
    get_playlist_ids(item)

current counts of ids: 35
current counts of ids: 70
current counts of ids: 105
current counts of ids: 140
current counts of ids: 175
current counts of ids: 210
current counts of ids: 245
current counts of ids: 280
current counts of ids: 315
current counts of ids: 350
current counts of ids: 385
current counts of ids: 420
current counts of ids: 455
current counts of ids: 490
current counts of ids: 525
current counts of ids: 560
current counts of ids: 595
current counts of ids: 630
current counts of ids: 665
current counts of ids: 700
current counts of ids: 735
current counts of ids: 770
current counts of ids: 805
current counts of ids: 840
current counts of ids: 875
current counts of ids: 910
current counts of ids: 945
current counts of ids: 980
current counts of ids: 1015
current counts of ids: 1050
current counts of ids: 1085
current counts of ids: 1120
current counts of ids: 1155
current counts of ids: 1190
current counts of ids: 1225
current counts of ids: 1260
current counts of ids:

In [272]:
len(play_list_ids)

89761

In [271]:
# 将可用ip写入到json文件
filename = 'playlist_ids.json'
with open(filename, 'w') as fw:
    json.dump(play_list_ids, fw)

## 获取代理ip并构建ip池

In [35]:
ips_file = 'valid_ids.json'
# 读取json中数据
with open(ips_file) as f:
    current_ip_list = json.load(f)

In [36]:
len(current_ip_list)

658

In [24]:
random_ip = random.choice(current_ip_list)
random_ip

'http://27.206.78.131:9000'

In [25]:
ip_list = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

# 获取ip池
def get_ip(index):
    #路径
    url = 'https://www.kuaidaili.com/free/inha/' + str(index)
    proxies = {}
    ip = random.choice(current_ip_list)
    if ip.split(":")[0] == 'http':
        proxies = {
            'http': ip
        }
    else:
        proxies = {
            'https': ip
        }
    #请求
    response = rq.get(url=url,headers=headers, proxies=proxies)
    #设置编码
    response.encoding = response.apparent_encoding
    response = response.text

    response = etree.HTML(response)

    tr_list = response.xpath('//*[@id="list"]/table/tbody/tr')
    for i in tr_list:
        ip = i.xpath('./td[1]/text()')[0]
        port = i.xpath('./td[2]/text()')[0]
        # 协议
        agreement = str(i.xpath('./td[4]/text()')[0])
        agreement = agreement.lower()
        # 拼装完整ip
        ip = agreement + '://' + ip + ':' + port
        ip_list.append(ip)
    print("count of ip: " + str(len(ip_list)))

In [26]:
for i in range(1, 60):
    get_ip(i)
    random_time = random.random()
    time.sleep(random_time*2)

count of ip: 15
count of ip: 15
count of ip: 30
count of ip: 30
count of ip: 30
count of ip: 45
count of ip: 45
count of ip: 60
count of ip: 75
count of ip: 90
count of ip: 105
count of ip: 105
count of ip: 120
count of ip: 120
count of ip: 135
count of ip: 135
count of ip: 150
count of ip: 150
count of ip: 165
count of ip: 180
count of ip: 195
count of ip: 210
count of ip: 225
count of ip: 240
count of ip: 255
count of ip: 255
count of ip: 270
count of ip: 285
count of ip: 285
count of ip: 300
count of ip: 315
count of ip: 315
count of ip: 330
count of ip: 345
count of ip: 345
count of ip: 360
count of ip: 375
count of ip: 390
count of ip: 405
count of ip: 420
count of ip: 435
count of ip: 450
count of ip: 465
count of ip: 480
count of ip: 495
count of ip: 510
count of ip: 525
count of ip: 525
count of ip: 540
count of ip: 555
count of ip: 570
count of ip: 570
count of ip: 570
count of ip: 585
count of ip: 600
count of ip: 615
count of ip: 630
count of ip: 645
count of ip: 660


In [27]:
test_ip_list = ip_list

In [28]:
len(test_ip_list)

660

In [29]:
valid_ip_list = []
#ip测试网站
url = 'https://www.ip.cn/'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0'
}
# 测试可用的ip
def ip_test(ip):
    try:
        proxies = {}
        if ip.split(":")[0] == 'http':
            proxies = {
                'http': ip
            }
        else:
            proxies = {
                'https': ip
            }
        response = rq.get(url=url, headers=headers, proxies=proxies, timeout=3)
        valid_ip_list.append(ip)
        print(ip + "可用")
    except:
        print(ip + "不可用")

In [30]:
from multiprocessing.dummy import Pool

In [31]:
pool = Pool(4)

In [32]:
pool.map(ip_test, test_ip_list)

http://182.105.201.40:9000可用
http://60.168.206.207:1133可用
http://175.44.109.239:9999可用
http://49.89.143.102:9999可用
http://110.243.3.38:9999可用
http://117.91.164.118:9999可用
http://113.121.39.137:9999可用
http://175.42.68.192:9999可用
http://175.44.108.75:9999可用
http://58.22.177.50:9999可用
http://175.42.123.114:9999可用
http://115.221.240.127:9999可用
http://36.248.132.25:9999可用
http://114.239.0.109:9999可用
http://183.166.70.196:9999可用
http://49.89.103.248:9999可用
http://171.35.173.180:9999可用
http://113.121.47.253:9999可用
http://175.42.122.236:9999可用
http://58.22.177.204:9999可用
http://49.86.26.166:9999可用
http://49.89.84.158:9999可用
http://183.166.97.131:9999可用
http://175.42.129.42:9999可用
http://115.218.3.214:9000可用
http://175.43.58.41:9999可用
http://175.42.123.175:9999可用
http://220.191.45.136:9000可用
http://60.169.134.220:9999可用
http://113.121.37.248:9999可用
http://117.64.237.133:1133可用
http://119.119.109.202:9000可用
http://175.44.109.202:9999可用
http://117.69.153.114:9999可用
http://175.42.158.150:9999可用
ht

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [33]:
print("总共爬取%s个ip，可用ip为：%s，不可用ip为：%s"%(len(test_ip_list),len(valid_ip_list),len(test_ip_list)-len(valid_ip_list)))

总共爬取660个ip，可用ip为：658，不可用ip为：2


In [34]:
# 将可用ip写入到json文件
filename = 'valid_ids.json'
with open(filename, 'w') as fw:
    json.dump(valid_ip_list, fw)

In [254]:
# 读取json中数据
with open(filename) as f:
    the_list = json.load(f)

# 根据歌单id爬取歌单详细数据

In [37]:
import pymongo
import threading

In [38]:
# 加载所有歌单id
filename = 'playlist_ids.json'
with open(filename) as f:
    all_playlist_ids = json.load(f)

In [39]:
len(all_playlist_ids)

89761

In [40]:
class process_info:
    # 处理失败的歌单id
    failed_parse_ids = []

In [41]:
# 请求头
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

In [63]:
# 数据库连接与选择
myclient = pymongo.MongoClient("mongodb://192.168.206.100:27017/")
springTest_db = myclient["springTest"]
mycol = springTest_db["test_play_list"]

In [43]:
# 将数据放入mongodb中
def save_to_mongodb(info_dict):
    try:
        # 判断数据库是否已有该歌单
        result_count = mycol.count_documents({'_id':info_dict.get('_id')})
        if result_count == 0:
            mycol.insert_one(info_dict)
    except Exception as e:
        print("mongdb Error: " + e)

In [44]:
# 处理歌单详细信息的json
def parse_info_json(str_json, playlist_id):
    # 将json转换为dict
    data_dict = json.loads(str_json)
    # 获取所有的歌曲id
    track_ids = data_dict.get("playlist").get("trackIds")
    song_ids = []
    for track in track_ids:
        song_id = track.get('id')
        song_ids.append(song_id)
    # 歌单的标签列表
    tags_list = data_dict.get("playlist").get("tags")
    # 歌单名称
    name = data_dict.get("playlist").get("name")
    # 歌单订阅数量
    subscribedCount = data_dict.get("playlist").get("subscribedCount")
    # 歌单播放数量
    playCount = data_dict.get("playlist").get("playCount")
    # 将数据封装成dict
    playlist_info_dict = {
        "_id": playlist_id,
        "song_ids":song_ids,
        "tags_list": tags_list,
        "name": name,
        "subscribedCount": subscribedCount,
        "playCount": playCount,
    }
    return playlist_info_dict

In [60]:
#获取首页该分类下面的歌单id
def get_playlist_info(playlist_id):
    start_url='https://autumnfish.cn/playlist/detail?id=' + playlist_id
    try:        
        # 随机抽取ip
        ip = random.choice(current_ip_list)
        proxies = {}
        if ip.split(":")[0] == 'http':
            proxies = {
                'http': ip
            }
        else:
            proxies = {
                'https': ip
            }
        # 请求API获得json格式的字符串
        str_json = rq.get(url=start_url, proxies=proxies, headers=headers)
        # 解析数据
        playlist_info_dict = parse_info_json(str_json.text, playlist_id)
        # 将数据放入mongodb中
        save_to_mongodb(playlist_info_dict)
        # 随机间隔时间
#         random_time = random.random()
#         time.sleep(random_time * 2)
    except Exception as e:
        # 将处理失败的id添加入相应列表
        process_info.failed_parse_ids.append(playlist_id)
        print('Error: ' + str(e))

In [53]:
from tqdm import tqdm, trange

In [64]:
len(all_playlist_ids)

89761

In [ ]:
with trange(len(all_playlist_ids)) as t:
    for i, item in enumerate(all_playlist_ids):
        get_playlist_info(item)
        t.set_description('Process %i' % i)  # 描述显示在左边
        t.update(1)

Process 2078:   2%|█▍                                                          | 2079/89761 [16:05<12:19:40,  1.98it/s]

In [62]:
start_time = time.time()
get_playlist_info(all_playlist_ids[4])
end_time = time.time()
print("运行时间："+str(end_time-start_time))

运行时间：0.4669215679168701
